In [8]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

# ----------------------------------------------------------
# CONFIG
# ----------------------------------------------------------

excel_path = "data/Fallzahlen_HZ.xlsx"
geo_path   = "data/lor_bezirksregionen_2021.geojson"
output_path = "data/berlin_crime_final.csv"

# ----------------------------------------------------------
# 1. Detect sheets
# ----------------------------------------------------------

xls = pd.ExcelFile(excel_path)
fall_sheets = [s for s in xls.sheet_names if s.startswith("Fallzahlen_")]
hz_sheets   = [s for s in xls.sheet_names if s.startswith("HZ_")]

print("Fallzahlen sheets:", fall_sheets)
print("HZ sheets:", hz_sheets)

# ----------------------------------------------------------
# 2. Bezirk mapping from LOR prefix
# ----------------------------------------------------------

bezirk_map = {
    1: "Mitte",
    2: "Friedrichshain-Kreuzberg",
    3: "Pankow",
    4: "Charlottenburg-Wilmersdorf",
    5: "Spandau",
    6: "Steglitz-Zehlendorf",
    7: "Tempelhof-Schöneberg",
    8: "Neukölln",
    9: "Treptow-Köpenick",
    10: "Marzahn-Hellersdorf",
    11: "Lichtenberg",
    12: "Reinickendorf"
}

# ----------------------------------------------------------
# 3. Clean a single sheet (Fallzahlen or HZ)
# ----------------------------------------------------------

def clean_sheet(df, year, value_colname):

    # Standardize columns
    df.columns = (
        df.columns
        .astype(str)
        .str.replace("\n", " ")
        .str.replace("-", " ")
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    # Rename first two columns
    df = df.rename(columns={
        df.columns[0]: "LOR_ID",
        df.columns[1]: "Bezirksregion"
    })

    # Convert LOR ID
    df["LOR_ID"] = pd.to_numeric(df["LOR_ID"], errors="coerce")

    # FIX: remove non-numeric rows to avoid IntCastingNaNError
    df = df[df["LOR_ID"].notna()].copy()

    # Bezirk from LOR prefix
    df["Bezirk_NR"] = (df["LOR_ID"] // 10000).astype(int)
    df["Bezirk"] = df["Bezirk_NR"].map(bezirk_map)

    # Identify crime-type columns
    non_crime = ["LOR_ID", "Bezirksregion", "Bezirk", "Bezirk_NR"]
    crime_cols = [c for c in df.columns if c not in non_crime]

    # Melt to long format
    df_long = df.melt(
        id_vars=["LOR_ID", "Bezirk", "Bezirksregion"],
        value_vars=crime_cols,
        var_name="CrimeType",
        value_name=value_colname
    )

    df_long["year"] = year
    return df_long

# ----------------------------------------------------------
# 4. Load & clean all Fallzahlen and HZ sheets
# ----------------------------------------------------------

fall_list = []
hz_list = []

for sheet in fall_sheets:
    year = int(sheet.replace("Fallzahlen_", ""))
    df = pd.read_excel(excel_path, sheet_name=sheet)
    fall_list.append(clean_sheet(df, year, "Fallzahl"))

for sheet in hz_sheets:
    year = int(sheet.replace("HZ_", ""))
    df = pd.read_excel(excel_path, sheet_name=sheet)
    hz_list.append(clean_sheet(df, year, "HZ"))

fall_df = pd.concat(fall_list, ignore_index=True)
hz_df   = pd.concat(hz_list, ignore_index=True)

# ----------------------------------------------------------
# 5. Merge Fallzahlen + HZ
# ----------------------------------------------------------

crime_df = fall_df.merge(
    hz_df,
    on=["year", "LOR_ID", "Bezirk", "Bezirksregion", "CrimeType"],
    how="left"
)

# ----------------------------------------------------------
# 6. FILTER: keep only true Bezirksregionen (not districts or totals)
# ----------------------------------------------------------
# District totals end in 0000
# Not-assignable summaries end in 9900
# Entire-city summaries have LOR_ID >= 900000

crime_df = crime_df[
    (crime_df["LOR_ID"] % 10000 != 0) &     # remove 010000, 020000 ...
    (crime_df["LOR_ID"] % 10000 != 9900) &  # remove 019900, 029900 ...
    (crime_df["LOR_ID"] < 900000)           # remove 999900, 999999
]

# ----------------------------------------------------------
# 7. Merge with GeoJSON (LOR boundaries)
# ----------------------------------------------------------

gdf = gpd.read_file(geo_path)

# GeoJSON key is BZR_ID
gdf["LOR_ID"] = gdf["BZR_ID"].astype(int)

# Convert CRS to WGS84 for Power BI
gdf = gdf.to_crs(4326)

# Merge spatial geometry
final = crime_df.merge(
    gdf[["LOR_ID", "geometry"]],
    on="LOR_ID",
    how="left"
)

final = gpd.GeoDataFrame(final, geometry="geometry")

# ----------------------------------------------------------
# 8. Add centroid coordinates (for Power BI mapping)
# ----------------------------------------------------------

final["lon"] = final.geometry.centroid.x
final["lat"] = final.geometry.centroid.y

# ----------------------------------------------------------
# 9. Export clean dataset
# ----------------------------------------------------------

final.to_csv(output_path, index=False)
print("✔ CLEAN DATA READY →", output_path)
print(final.head())
print("Rows:", len(final))


Fallzahlen sheets: ['Fallzahlen_2015', 'Fallzahlen_2016', 'Fallzahlen_2017', 'Fallzahlen_2018', 'Fallzahlen_2019', 'Fallzahlen_2020', 'Fallzahlen_2021', 'Fallzahlen_2022', 'Fallzahlen_2023', 'Fallzahlen_2024']
HZ sheets: ['HZ_2015', 'HZ_2016', 'HZ_2017', 'HZ_2018', 'HZ_2019', 'HZ_2020', 'HZ_2021', 'HZ_2022', 'HZ_2023', 'HZ_2024']


/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_31620/3332891007.py:159: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  final["lon"] = final.geometry.centroid.x
/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_31620/3332891007.py:160: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  final["lat"] = final.geometry.centroid.y


✔ CLEAN DATA READY → data/berlin_crime_final.csv
    LOR_ID Bezirk      Bezirksregion             CrimeType Fallzahl  year  \
0  11001.0  Mitte     Tiergarten Süd  Straftaten insgesamt     5475  2015   
1  11002.0  Mitte  Regierungsviertel  Straftaten insgesamt    10313  2015   
2  11003.0  Mitte     Alexanderplatz  Straftaten insgesamt    23524  2015   
3  11004.0  Mitte  Brunnenstraße Süd  Straftaten insgesamt     4583  2015   
4  12005.0  Mitte        Moabit West  Straftaten insgesamt     7428  2015   

       HZ                                           geometry        lon  \
0   38575  MULTIPOLYGON (((13.3733 52.50373, 13.37329 52....  13.358181   
1  102068  MULTIPOLYGON (((13.38997 52.50757, 13.38995 52...  13.390625   
2   45938  MULTIPOLYGON (((13.42303 52.51212, 13.42308 52...  13.402246   
3   16175  MULTIPOLYGON (((13.38783 52.53339, 13.38788 52...  13.393361   
4   16612  MULTIPOLYGON (((13.34344 52.52602, 13.34343 52...  13.332072   

         lat  
0  52.510452  
1  52.5